In [1]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout,
    QTableWidget, QTableWidgetItem, QPushButton, QSplitter, QListWidget,
    QMessageBox, QStatusBar
)
from PyQt5.QtCore import Qt, QUrl, QTimer
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEnginePage
import pandas as pd  # pandas import 추가

class CustomWebEnginePage(QWebEnginePage):
    def javaScriptConsoleMessage(self, level, message, lineNumber, sourceId):
        print(f"JavaScript console message: {level=}, {message=}, {lineNumber=}, {sourceId=}")

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.login_data = self.load_login_data()
        self.initUI()
        self.login_attempt_count = 0
        self.is_login_process = False  # 로그인 프로세스 상태 변수 추가

    def initUI(self):
        self.setWindowTitle("Sample Crawler UI")
        self.setGeometry(100, 100, 1200, 800)

        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)
        self.status_bar.showMessage("Ready")

        main_widget = QWidget()
        main_layout = QVBoxLayout(main_widget)

        top_layout = QHBoxLayout()

        load_sites_button = QPushButton("사이트 목록 불러오기")
        load_sites_button.clicked.connect(self.load_sites)

        open_site_button = QPushButton("선택된 사이트 열기")
        open_site_button.clicked.connect(self.open_selected_site)

        login_button = QPushButton("로그인")
        login_button.clicked.connect(self.login_to_site)

        top_layout.addWidget(load_sites_button)
        top_layout.addWidget(open_site_button)
        top_layout.addWidget(login_button)

        main_layout.addLayout(top_layout)

        self.site_list_widget = QListWidget()
        main_layout.addWidget(self.site_list_widget)

        h_splitter = QSplitter(Qt.Horizontal)

        self.web_view = QWebEngineView()
        self.web_view.setPage(CustomWebEnginePage(self.web_view))
        self.web_view.loadStarted.connect(self.on_load_started)
        self.web_view.loadProgress.connect(self.on_load_progress)
        self.web_view.loadFinished.connect(self.on_load_finished)
        h_splitter.addWidget(self.web_view)

        self.table_widget = QTableWidget()
        self.table_widget.setRowCount(10)

        header_labels = ["원상품코드", "상품코드", "원상품명", "변경상품명", "스마트스토어전용상품명", "쿠팡전용상품명",
                         "상품명길이", "판매단위", "마이링크", "주키워드", "부키워드", "세키워드", "네이버쇼핑사전등록태그",
                         "스마트스토어 태그", "실제등록상품명", "등록불가단어", "원산지", "제조사", "수입사", "브랜드",
                         "모델명", "과세여부", "나이제한", "제조일", "유효기간", "홍보문구", "공급원가", "가격기준",
                         "가격요율", "판매금액", "우선판매금액", "옵션입력", "옵션명", "옵션정보", "옵션개수", "옵션분할명",
                         "재고수량", "원대표이미지", "원추가이미지1", "원추가이미지2", "원추가이미지3", "원추가이미지4",
                         "원추가이미지5", "대표이미지", "추가이미지1", "추가이미지2", "추가이미지3", "추가이미지4",
                         "추가이미지5", "원상세설명", "상세설명", "상세설명특이사항상단", "상세설명특이사항하단", "최종상세설명",
                         "옥션", "지마켓", "11번가", "11번가상품속성명", "11번가상품세부속성명", "11번가추천옵션명", "인터파크",
                         "스마트스토어", "쿠팡", "쿠팡추천옵션명", "쿠팡검색옵션명", "옥션CAT", "지마켓CAT", "11번가CAT",
                         "인터파크CAT", "스마트스토어CAT", "쿠팡CAT", "ESM옥션카테고리", "ESM지마켓카테고리", "ESM옥션추천옵션명",
                         "ESM지마켓추천옵션명", "티몬", "위메프", "롯데ON", "롯데ON구매옵션", "톡스토어", "카페24", "멸치쇼핑",
                         "72TIME", "에이블리", "특이사항", "인증대상여부", "인증품목", "인증번호", "인증기관", "인증상호",
                         "인증일자", "인증제조국", "인증제조사", "인증모델명", "인증상태", "최저가준수", "소비자가", "상품상태",
                         "단위", "규격", "ISBN", "최소구매수량", "이셀러스요약정보상품군코드", "스마트스토어상품속성", "쿠팡검색필터",
                         "쿠팡전용필수옵션", "공급사카테고리", "수집1", "수집2", "수집3", "수집4", "수집5", "수집6", "수집7",
                         "수집8", "수집9", "수집10", "수집배송비", "편집1", "편집2", "편집3", "편집4", "편집5", "편집6",
                         "편집7", "편집8", "편집9", "편집10", "단독옵션일괄변경여부", "비고", "경로코드", "원본경로"]
        self.table_widget.setColumnCount(len(header_labels))
        self.table_widget.setHorizontalHeaderLabels(header_labels)

        for row in range(10):
            self.table_widget.setItem(row, 0, QTableWidgetItem(f"상품명 {row+1}"))
            self.table_widget.setItem(row, 1, QTableWidgetItem(str((row+1)*1000)))
            self.table_widget.setItem(row, 2, QTableWidgetItem(str(row+1)))
            self.table_widget.setItem(row, 3, QTableWidgetItem("기타 정보"))

        h_splitter.addWidget(self.table_widget)

        v_splitter = QSplitter(Qt.Vertical)
        v_splitter.addWidget(self.site_list_widget)
        v_splitter.addWidget(h_splitter)

        v_splitter.setSizes([int(self.height() * 0.2), int(self.height() * 0.8)])
        h_splitter.setSizes([int(self.width() * 0.7), int(self.width() * 0.3)])

        main_layout.addWidget(v_splitter)
        self.setCentralWidget(main_widget)

    def load_login_data(self):
        try:
            df = pd.read_excel("login_data.xlsx")
            login_dict = {}
            for index, row in df.iterrows():
                site_url = row['사이트목록']
                login_dict[site_url] = {
                    '아이디': row['아이디'],
                    '비밀번호': row['비밀번호'],
                    '로그인페이지': row['로그인페이지'] if pd.notnull(row['로그인페이지']) else None,
                    '로그인후페이지': row['로그인후페이지'] if pd.notnull(row['로그인후페이지']) else None
                }
            return login_dict
        except FileNotFoundError:
            QMessageBox.critical(self, "오류", "login_data.xlsx 파일을 찾을 수 없습니다.")
            return {}
        except Exception as e:
            QMessageBox.critical(self, "오류", f"login_data.xlsx 파일을 읽는 중 오류 발생: {e}")
            return {}

    def load_sites(self):
        self.site_list_widget.clear()
        try:
            with open("site_list.txt", "r", encoding="utf-8") as f:
                for line in f:
                    site = line.strip()
                    if site:
                        self.site_list_widget.addItem(site)
            if not self.site_list_widget.count():
                QMessageBox.information(self, "알림", "사이트 목록 파일에 사이트 정보가 없습니다.")
        except FileNotFoundError:
            QMessageBox.critical(self, "오류", "site_list.txt 파일을 찾을 수 없습니다.")
        except Exception as e:
            QMessageBox.critical(self, "오류", f"파일을 읽는 중 오류가 발생했습니다: {e}")

    def open_selected_site(self):
        current_item = self.site_list_widget.currentItem()
        if current_item:
            url_text = current_item.text().strip()
            if not url_text.startswith(("http://", "https://")):
                url_text = "https://" + url_text
            url = QUrl(url_text)
            if url.isValid():
                print("Loading site:", url_text)
                self.web_view.load(url)
            else:
                QMessageBox.warning(self, "경고", f"유효하지 않은 URL입니다: {url_text}")
        else:
            QMessageBox.information(self, "알림", "사이트를 선택해주세요.")

    def login_to_site(self):
        current_item = self.site_list_widget.currentItem()
        if not current_item:
            QMessageBox.information(self, "알림", "로그인할 사이트를 먼저 선택해주세요.")
            return

        site_url_base = current_item.text().strip()
        if not site_url_base.startswith(("http://", "https://")):
            site_url_base = "https://" + site_url_base

        login_info = self.login_data.get(site_url_base)
        if not login_info:
            QMessageBox.warning(self, "경고", f"{site_url_base}에 대한 로그인 정보가 login_data.xlsx에 없습니다.")
            return

        login_page_url = login_info.get('로그인페이지')
        if not login_page_url:
            QMessageBox.warning(self, "경고", f"{site_url_base}의 로그인 페이지 URL이 login_data.xlsx에 설정되지 않았습니다.")
            return

        self.web_view.load(QUrl(login_page_url))
        self.status_bar.showMessage(f"로그인 페이지 {login_page_url} 로딩 중... (Attempt {self.login_attempt_count + 1})")
        self.login_attempt_count += 1
        self.is_login_process = True # 로그인 프로세스 시작 상태 설정

        self.current_login_info = login_info
        self.web_view.login_site_url_base = site_url_base


    def on_load_started(self):
        self.status_bar.showMessage("Loading...")
        self.setWindowTitle(f"Loading... - Sample Crawler UI")

    def on_load_progress(self, progress):
        self.status_bar.showMessage(f"Loading... {progress}% ")

    def on_load_finished(self, ok):
        if ok:
            url = self.web_view.url().toString()
            self.setWindowTitle(f"{self.web_view.title()} - Sample Crawler UI")

            print(f"** on_load_finished: URL loaded: {url}, Attempt: {self.login_attempt_count}, Login Process: {self.is_login_process} **")

            if self.is_login_process: # 로그인 프로세스 중일 때만 스크립트 실행
                login_info = self.current_login_info
                site_url_base = self.web_view.login_site_url_base

                print(f"** on_load_finished: Attempting login (Attempt: {self.login_attempt_count}) **")

                if login_info:
                    username = login_info['아이디']
                    password = login_info['비밀번호']

                    script = f"""
                        (function() {{
                            let usernameField = document.querySelector('#member_id');
                            let passwordField = document.querySelector('#member_passwd');
                            let loginButton = document.querySelector('a.-btn.-block.-xl.-black');

                            console.log('Login attempt: {self.login_attempt_count}');
                            console.log('Username field selector: #member_id, Password field selector: #member_passwd, Login button selector: a.-btn.-block.-xl.-black');

                            if (usernameField) {{
                                console.log('Username field found:', usernameField);
                                usernameField.value = '{username}';
                                console.log('Username field value set.');
                            }} else {{
                                console.error('Username field not found for {site_url_base}');
                            }}

                            if (passwordField) {{
                                console.log('Password field found:', passwordField);
                                passwordField.value = '{password}';
                                console.log('Password field value set.');
                            }} else {{
                                console.error('Password field not found for {site_url_base}');
                            }}

                            if (loginButton) {{
                                let loginFunc = loginButton.onclick;
                                if (loginFunc) {{
                                    console.log('Login button onclick function found:', loginFunc);
                                    loginFunc.call(loginButton);
                                    console.log('Login button onclick function executed.');
                                }} else {{
                                    console.warn('Login button onclick function NOT found. Trying button click fallback.');
                                    loginButton.click();
                                    console.log('Login button clicked (fallback).');
                                }}

                            }} else {{
                                console.warn('Login button not found for {site_url_base}. Form submission might be required.');
                            }}
                        }})();
                    """
                    # 딜레이 후 JavaScript 실행 (1초)
                    QTimer.singleShot(1000, lambda: self.web_view.page().runJavaScript(script)) # Delayed execution
                    self.status_bar.showMessage(f"{site_url_base} 로그인 시도 중... (Attempt {self.login_attempt_count}) - Delayed") # Delayed status message
                    print(f"** on_load_finished: JavaScript login script execution DELAYED for {site_url_base} (Attempt: {self.login_attempt_count}) **") # Delayed execution log

                    self.is_login_process = False # 로그인 프로세스 종료 상태 설정
                    del self.current_login_info
                    del self.web_view.login_site_url_base
                else:
                    self.status_bar.showMessage("Load finished")
            else:
                self.status_bar.showMessage("Load finished")

        else:
            self.status_bar.showMessage("Load failed")
            self.setWindowTitle(f"Load Failed - Sample Crawler UI")
            QMessageBox.critical(self, "오류", "웹 페이지 로딩 실패.")


def main():
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()

JavaScript console message: level=2, message='Uncaught ReferenceError: AOS is not defined', lineNumber=1862, sourceId='https://domesin.co.kr/member/login.html'
JavaScript console message: level=1, message="Synchronous XMLHttpRequest on the main thread is deprecated because of its detrimental effects to the end user's experience. For more help, check https://xhr.spec.whatwg.org/.", lineNumber=4, sourceId='https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js'
JavaScript console message: level=0, message='ERROR: cannot decode, now / past', lineNumber=25734, sourceId='https://domesin.co.kr/ind-script/optimizer.php?filename=zVhbV-M2EH4Pfu3vMHTbnr5CWFoO4WwOge7zWB47wrJG1YXg_fUdO9mWbLDjSzinL7Ej6_s0o7lK8ZpKjC9-tnFmocQN2SK26ChYgfGzi-XF7zp6dj_FXfOCNCCK-PnvgLbaPT5Fv0XnA5H46tFqUG43EAmiQuJUlmDVKBUiFFGmR0NBSXDj0PzVk_1BcWMpZ_yeeVLw6CVPKHmW9jGK--blq_Trx2_X4CEqpR7Ao2TCKHzk9-MoZ6zUPutBTyLUYt2QLeekvSWl0B7HLVVwl8ZcWZnmPeQJXqrjszJLzU7NHClGkJ55MjMlNc4S0LqPXFsKQWVJuu_sV9AcTpp901j0_VdpcB5Lo9gu74PA

SystemExit: 0

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
